# AVANCE 1

Importar Librerias

In [5]:
import pandas as pd
import numpy as np

Definir ruta de archivo CSV

In [6]:
ruta_archivo = r"C:\Users\PC\Documents\HENRY\MOD 4\PROYECTO INTEGRADOS M4\BASE INFORMACION\data_latinoamerica.csv"

#Tamaño de cada chunk 
chunk_size = 100000



Definir países y fecha de corte

In [7]:
#Lista de países donde se expandirán
paises_objetivo = ['Colombia','Argentina','Chile', 'Mexico', 'Peru', 'Brasil']

#Fecha de corte
fecha_corte = '2021-01-01'


Leer y filtrar por chunk

In [8]:
#Lista para guardar los datos filtrados
datos_filtrados =[]

#Leer el archivo por partes
for chunk in pd.read_csv(ruta_archivo, chunksize = chunk_size, parse_dates = ['date']):
    
    #filtrar por país y fecha
    chunk_filtrado = chunk[
        (chunk['country_name'].isin(paises_objetivo)) &
        (chunk['date'] > fecha_corte)
    ]
    datos_filtrados.append(chunk_filtrado)

#Unir todos los datos filtrados en un solo DataFrame
df = pd.concat(datos_filtrados,ignore_index = True)


Verificar cantidad de registros y columnas

In [9]:
print(f'✔️ El dataset filtrado tiene {df.shape[0]:,} registros y {df.shape[1]} columnas,')
df.head()


✔️ El dataset filtrado tiene 4,032,912 registros y 50 columnas,


,location_key,date,country_code,country_name,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_vaccine_doses_administered,population,...,population_largest_city,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,new_recovered,cumulative_recovered
0,AR,2021-01-02,AR,Argentina,7767.0,166.0,1670634.0,46688.0,20334.0,44938712.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,NaN,NaN
1,AR,2021-01-03,AR,Argentina,4934.0,157.0,1675568.0,46845.0,20529.0,44938712.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,NaN,NaN
2,AR,2021-01-04,AR,Argentina,13953.0,157.0,1689521.0,47002.0,25164.0,44938712.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,NaN,NaN
3,AR,2021-01-05,AR,Argentina,14085.0,160.0,1703606.0,47162.0,29852.0,44938712.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,NaN,NaN
4,AR,2021-01-06,AR,Argentina,14496.0,131.0,1718102.0,47293.0,34162.0,44938712.0,...,15057273.0,2690269.0,55032.0,76.52,147.086,79.483,26.6,15.8,NaN,NaN


Completar valores faltantes por país (relleno por promedio de cada país)

In [10]:
#Llenar valores numéricos faltantes por promedio del país
columnas_numericas = df.select_dtypes(include=[np.number]).columns

for col in columnas_numericas:
    df[col] =df.groupby('country_name')[col].transform(lambda x: x.fillna(x.mean()))


"""¿Qué hace?
Detecta las columnas numéricas del dataset.
Para cada una, rellena los valores nulos (NaN) con el promedio del país (groupby("nombre_país")).
Por ejemplo: si la columna pib tiene valores nulos en México, los rellena con el promedio de pib en México.
"""

'¿Qué hace?\nDetecta las columnas numéricas del dataset.\nPara cada una, rellena los valores nulos (NaN) con el promedio del país (groupby("nombre_país")).\nPor ejemplo: si la columna pib tiene valores nulos en México, los rellena con el promedio de pib en México.\n'

Limpiar valores restantes

In [11]:
#Rellenar con método hacia adelante y hacia atras por país
df = df.sort_values(by=['country_name','date'])
df = df.groupby ('country_name').apply (lambda group: group.ffill().bfill()).reset_index(drop=True)

#Verificar si quedan nulos
nulos_totales = df.isnull().sum().sum()
print(f'🔍 Total de valores nulos despues de limpieza:{nulos_totales}')


C:\Users\PC\AppData\Local\Temp\ipykernel_1196\3673006257.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby ('country_name').apply (lambda group: group.ffill().bfill()).reset_index(drop=True)


🔍 Total de valores nulos despues de limpieza:6396624


Revision de dataset

In [12]:
#Ver cantidad de columnas y filas
print(f"✅ El DataFrame final tiene {df.shape[0]:,} filas y {df.shape[1]} columnas.")

#Ver muestra de contenido
df.head() 

#Ver columnas y tipos de datos
df.info()



✅ El DataFrame final tiene 4,032,912 filas y 50 columnas.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032912 entries, 0 to 4032911
Data columns (total 50 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   location_key                           object        
 1   date                                   datetime64[ns]
 2   country_code                           object        
 3   country_name                           object        
 4   new_confirmed                          float64       
 5   new_deceased                           float64       
 6   cumulative_confirmed                   float64       
 7   cumulative_deceased                    float64       
 8   cumulative_vaccine_doses_administered  float64       
 9   population                             float64       
 10  population_male                        float64       
 11  population_female                      float64       
 12

Analisis exploratorio basico: Incidencias de COVID

In [13]:
#Ver columnas clave relacionadas con COVID
columnas_covid = [
    'new_confirmed', 'new_deceased', 'new_recovered',
    'cumulative_confirmed', 'cumulative_deceased', 'cumulative_recovered'
]

#Estadisticas basicas por pais
df_covid = df.groupby('country_name')[columnas_covid].sum().sort_values(by='cumulative_confirmed',ascending=False)

pd.options.display.float_format = '{:,.0f}'.format




In [14]:
# Mostrar los primeros 10 países con más casos acumulados
print("📊 Totales acumulados por país:")
df_covid.head(10)

📊 Totales acumulados por país:


,new_confirmed,new_deceased,new_recovered,cumulative_confirmed,cumulative_deceased,cumulative_recovered
country_name,,,,,,
Mexico,"125,532,393","2,470,365",0,"26,524,574,382","1,128,294,132",0
Colombia,"40,325,309","1,037,162","38,100,751","18,926,845,373","520,740,448","13,455,268,402"
Peru,"46,897,314","2,117,947",0,"18,476,724,816","1,507,832,723",0
Argentina,"29,917,310","323,484",0,"12,159,879,088","220,825,661",0
Chile,"37,212,360","463,261",0,"10,027,702,078","182,060,694",0


Ver resumen estadístico 

In [15]:
print("📐 Dimensiones finales del dataset:")
print(f"🔢 Filas: {df.shape[0]:,}")
print(f"🔢 Columnas: {df.shape[1]}")

print("\n👀 Vista previa:")
display(df.head())

print("\n📋 Tipos de datos:")
df.info()

print("\n📊 Estadísticas básicas:")
df.describe()


📐 Dimensiones finales del dataset:
🔢 Filas: 4,032,912
🔢 Columnas: 50

👀 Vista previa:


,location_key,date,country_code,country_name,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_vaccine_doses_administered,population,...,population_largest_city,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,new_recovered,cumulative_recovered
0,AR,2021-01-02,AR,Argentina,"7,767",166,"1,670,634","46,688","20,334","44,938,712",...,"15,057,273","2,690,269","55,032",77,147,79,27,16,NaN,NaN
1,AR_A,2021-01-02,AR,Argentina,20,2,"22,367","1,169","1,294","1,406,584",...,"15,057,273","2,690,269","55,032",77,147,79,27,16,NaN,NaN
2,AR_A_007,2021-01-02,AR,Argentina,1,0,700,23,"168,819","57,411",...,"15,057,273","2,690,269","55,032",77,147,79,27,16,NaN,NaN
3,AR_A_014,2021-01-02,AR,Argentina,88,1,"35,756",649,2,"7,315",...,"15,057,273","2,690,269","55,032",77,147,79,27,16,NaN,NaN
4,AR_A_021,2021-01-02,AR,Argentina,0,0,589,20,"168,819","14,850",...,"15,057,273","2,690,269","55,032",77,147,79,27,16,NaN,NaN



📋 Tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032912 entries, 0 to 4032911
Data columns (total 50 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   location_key                           object        
 1   date                                   datetime64[ns]
 2   country_code                           object        
 3   country_name                           object        
 4   new_confirmed                          float64       
 5   new_deceased                           float64       
 6   cumulative_confirmed                   float64       
 7   cumulative_deceased                    float64       
 8   cumulative_vaccine_doses_administered  float64       
 9   population                             float64       
 10  population_male                        float64       
 11  population_female                      float64       
 12  population_rural                     

,date,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,cumulative_vaccine_doses_administered,population,population_male,population_female,population_rural,...,population_largest_city,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate,new_recovered,cumulative_recovered
count,4032912,"4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912",...,"4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","4,032,912","947,856","721,344"
mean,2021-11-09 12:00:00.000001024,69,2,"21,353",883,"45,643,166","135,866","324,120","340,259","13,873,556",...,"15,056,281","1,539,168","56,195",76,160,86,43,15,40,"18,653"
min,2021-01-02 00:00:00,-101,-53,0,0,0,4,12,29,"2,341,903",...,"6,723,516","709,418","12,027",75,108,59,25,12,0,0
25%,2021-06-06 18:00:00,40,1,"15,576",722,"2,018,681","3,843","1,994","2,036","7,120,114",...,"10,554,712","1,256,339","16,425",75,146,79,37,13,0,"3,550"
50%,2021-11-09 12:00:00,56,2,"17,017",724,"40,871,661","13,434","7,527","7,726","9,512,141",...,"10,779,376","1,256,339","36,132",77,152,85,37,16,53,"18,653"
75%,2022-04-14 06:00:00,81,2,"17,017","1,271","80,829,641","47,932","58,173","60,983","24,948,670",...,"21,671,908","1,831,423","102,418",77,184,96,64,16,53,"18,653"
max,2022-09-17 00:00:00,"174,174","11,447","9,697,763","304,025","209,673,612","110,991,953","54,855,231","57,481,307","24,948,670",...,"21,671,908","2,690,269","102,418",80,184,96,64,16,"61,030","5,441,021"
std,NaN,477,11,"123,566","4,150","31,730,029","1,738,221","1,018,411","1,066,954","8,977,560",...,"5,481,620","492,885","38,376",1,22,10,14,2,322,"99,534"


Estadisticas descritivas con bucle for 

In [16]:
#Seleccionar solo columnas numéricas
columnas_numericas = df.select_dtypes (include = 'number').columns

#Calcular estadisticas descriptiva con bucle
for col in columnas_numericas:
    print(f'\n📊 Estadísticas para: {col}')
    print(f'🔹 Media: {df[col].mean():,.2f}')
    print(f'🔹 Mediana: {df[col].median():,.2f}')
    print(f'🔹 Mínimo: {df[col].min():,.2f}')
    print(f'🔹 Máximo: {df[col].max():,.2f}')
    print(f'🔹 Desviación estándar: {df[col].std():,.2f}')
    print(f'🔹 Varianza: {df[col].var():,.2f}')


📊 Estadísticas para: new_confirmed
🔹 Media: 69.40
🔹 Mediana: 55.90
🔹 Mínimo: -101.00
🔹 Máximo: 174,174.00
🔹 Desviación estándar: 476.93
🔹 Varianza: 227,463.49

📊 Estadísticas para: new_deceased
🔹 Media: 1.59
🔹 Mediana: 1.58
🔹 Mínimo: -53.00
🔹 Máximo: 11,447.00
🔹 Desviación estándar: 10.69
🔹 Varianza: 114.38

📊 Estadísticas para: cumulative_confirmed
🔹 Media: 21,353.24
🔹 Mediana: 17,016.55
🔹 Mínimo: 0.00
🔹 Máximo: 9,697,763.00
🔹 Desviación estándar: 123,565.80
🔹 Varianza: 15,268,505,825.23

📊 Estadísticas para: cumulative_deceased
🔹 Media: 882.68
🔹 Mediana: 723.84
🔹 Mínimo: 0.00
🔹 Máximo: 304,025.00
🔹 Desviación estándar: 4,149.82
🔹 Varianza: 17,221,019.80

📊 Estadísticas para: cumulative_vaccine_doses_administered
🔹 Media: 45,643,166.09
🔹 Mediana: 40,871,660.99
🔹 Mínimo: 0.00
🔹 Máximo: 209,673,612.00
🔹 Desviación estándar: 31,730,028.68
🔹 Varianza: 1,006,794,719,962,522.38

📊 Estadísticas para: population
🔹 Media: 135,865.66
🔹 Mediana: 13,434.00
🔹 Mínimo: 4.00
🔹 Máximo: 110,991,953.00

Respuesta a las preguntas en el punto 11:
¿Qué implican estas métricas y cómo ayudan en el análisis?
Estas métricas (media, mediana, desviación estándar, etc.) permiten:
Entender la distribución de los datos
Detectar valores atípicos
Comparar comportamientos entre países o periodos
Medir variabilidad o consistencia en los datos

¿Se muestran todas las estadísticas en todas las columnas durante el análisis?
No. Algunas funciones como df.describe() sólo calculan estadísticas para columnas numéricas, y no incluyen mediana, rango, ni varianza por defecto.

¿Cuál es la razón y cómo puede afectar eso la interpretación?
Esto ocurre porque describe() está diseñado para un resumen general rápido. No incluir mediana o rango puede ocultar:
Distribuciones asimétricas (por eso es clave usar también la mediana)
Casos con gran dispersión (varianza y rango ayudan a ver esto)
Lo anterior puede llevar a interpretar que los datos están más “balanceados” de lo que realmente están.


Crear una funcion personalizada
Objetivo: Función para obtener mediana, varianza y rango, y responder preguntas sobre dispersión.

In [17]:
def estadisticas_personalizadas(df,columnas):
    resultados = {}
    for col in columnas:
        mediana = df[col].median()
        varianza = df[col].var()
        rango = df[col].max() - df[col].min()
        resultados[col] = {
            'mediana': round(mediana,2),
            'varianza': round(varianza,2), 
            'rango':round(rango,2)
        }
    return pd.DataFrame(resultados).T   #Transpuesta mejor visualización

#Aplicar a columnas objetivo
columnas_objetivo = ['new_confirmed', 'new_deceased', 'population']
estadisticas_personalizadas(df,columnas_objetivo)
    

,mediana,varianza,rango
new_confirmed,56,"227,463","174,275"
new_deceased,2,114,"11,500"
population,"13,434","3,021,411,964,673","110,991,949"


Respuestas a las preguntas en el punto 12:
¿Qué representa la mediana?
La mediana es el valor central de una distribución. Es útil cuando hay valores extremos (outliers) porque no se ve afectada por ellos, a diferencia de la media. Es una medida de tendencia central.
Por eso, es muy útil para analizar datos como los de casos y muertes por COVID, donde puede haber picos anormales.

En este caso:
nuevos_confirmados: Mediana = 56 → En la mayoría de las observaciones, el número de nuevos confirmados ronda los 56.

nuevos_fallecidos: Mediana = 2 → En la mayoría de los días, los fallecidos nuevos por COVID fueron muy bajos.

población: Mediana = 13,434 → La mayoría de las regiones tienen poblaciones pequeñas comparadas con otras muy grandes (como Brasil o México), lo que indica alta desigualdad poblacional entre regiones.

¿Cómo varía la dispersión en términos de varianza y rango?
La varianza mide cuánto se dispersan los datos respecto a la media.
El rango indica la diferencia entre el valor más alto y el más bajo.
Si varianza y rango son altos, hay mucha variabilidad.
Si son bajos, los datos son más homogéneos o consistentes.

nuevos_confirmados	varianza 227,463	 rango 174,275
Tiene una varianza y rango muy altos, lo que indica que hay regiones o fechas con picos enormes de contagios comparado con otras.

nuevos_fallecidos	varianza 114	rango 11,500
Baja varianza, pero rango muy amplio → normalmente hay pocos fallecimientos, pero hay excepciones con cifras muy altas, que son outliers.

población	varianza 3,021,411,964,673	rango 110,991,949
Extremadamente dispersa. El rango es muy alto, lo que indica que algunas regiones tienen pocos habitantes y otras muchísimos.


¿Qué nos indica esto sobre la consistencia o variabilidad?
Un conjunto con:
Alta varianza y rango: datos poco consistentes, muy dispersos → puede dificultar análisis o predicciones.
Baja varianza y rango: datos estables, más confiables para análisis comparativo.

En población, aunque la media puede ser muy alta, la mayoría de los países (mediana) tienen poblaciones bajas → eso significa que la distribución es muy asimétrica.

En nuevos_confirmados, también pasa lo mismo: días normales con pocos casos, pero algunos días con miles, lo que afecta la interpretación de promedios.


In [18]:
df["location_key"].unique()

array(['AR', 'AR_A', 'AR_A_007', ..., 'PE_UCA_0304', 'PE_UCA_0305',
       'PE_UCA_0401'], shape=(6463,), dtype=object)

In [25]:
import pandas as pd

# 1. Ruta al archivo CSV (esto es un string, ¡no es el DataFrame!)
ruta_archivo = r"C:\Users\PC\Documents\HENRY\MOD 4\PROYECTO INTEGRADOS M4\BASE INFORMACION\data_latinoamerica.csv"

# 2. Leer el archivo y convertirlo en un DataFrame
data_latinoamerica = pd.read_csv(ruta_archivo)

# 3. Lista de países objetivo
paises_objetivo = ["Argentina",  "Brazil", "Colombia", "Chile", "Mexico", "Peru" ]

# 4. Filtrar por country_name
data_latinoamerica = data_latinoamerica[
    data_latinoamerica["country_name"].isin(paises_objetivo)
]

# 5. Filtrar por location_key
data_latinoamerica = data_latinoamerica[
    data_latinoamerica["location_key"].isin(["AR", "BR", "CO", "CL", "MX", "PE"])
]

# 6. Asegúrar de que la columna de fecha esté en formato datetime
data_latinoamerica["date"] = pd.to_datetime(data_latinoamerica["date"])

# 7. Filtrar por fecha desde el 2021-01-01 en adelante
data_latinoamerica = data_latinoamerica[data_latinoamerica["date"] >= "2020-12-31"]


# Guardar el archivo Final

In [26]:
data_latinoamerica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3756 entries, 365 to 10253876
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   location_key                           3756 non-null   object        
 1   date                                   3756 non-null   datetime64[ns]
 2   country_code                           3756 non-null   object        
 3   country_name                           3756 non-null   object        
 4   new_confirmed                          3735 non-null   float64       
 5   new_deceased                           3735 non-null   float64       
 6   cumulative_confirmed                   3735 non-null   float64       
 7   cumulative_deceased                    3735 non-null   float64       
 8   cumulative_vaccine_doses_administered  3162 non-null   float64       
 9   population                             3756 non-null   float64

In [27]:
print(data_latinoamerica.shape)


(3756, 50)


In [28]:
data_latinoamerica.to_csv('DatosFinalesFiltrados.csv', index=False)
print("✅ Archivo guardado como 'DatosFinalesFiltrados.csv'")


✅ Archivo guardado como 'DatosFinalesFiltrados.csv'


In [29]:
df_final = pd.read_csv('DatosFinalesFiltrados.csv')


In [30]:
# Obtener valores únicos de la columna 'country_name'
valores_unicos = df_final["country_name"].unique()

print(valores_unicos)


['Argentina' 'Brazil' 'Chile' 'Colombia' 'Mexico' 'Peru']
